In [32]:
import numpy as np
import matplotlib.pyplot as plt
import sys, os, cv2, lmdb, struct

caffe_root = '/home/heyanguang/caffecode/caffe/'
sys.path.insert(0, caffe_root + 'python')
os.chdir(caffe_root)
import caffe
from caffe.proto import caffe_pb2

def get_image_label(filename, read_count, start_location):
    f1 = open(filename, 'rb')
    buf1 = f1.read()
    
    image = np.zeros((read_count, 3, 32, 32)).astype('uint8')
    label = np.zeros(read_count).astype('uint8')
    image_offset = 3073 * start_location
    for i in range(read_count):
        im_la = []
        temp = struct.unpack_from('B', buf1, image_offset)
        label[i] = np.array(temp)
        image_offset += 1
        temp = struct.unpack_from('3072B', buf1, image_offset)
        image[i] = np.reshape(temp, (3, 32, 32)).astype('uint8')
        image_offset += 3072
    return image, label

# read_count = 10000
# start_location = 0
# image_label = get_image_label('data/cifar10/data_batch_1.bin', read_count, start_location)
# image_label = get_image_label('data/cifar10/test_batch.bin', read_count, start_location)
print image_label[0].shape
print len(image_label[0])

(10000, 3, 32, 32)
10000


In [29]:
a = image_label[0][10]
b = np.zeros((3, 256, 256)).astype('int')
print a.dtype
print b.dtype
print a.shape
for i in range(3):
    b[i] = cv2.resize(a[i], (256, 256), interpolation = cv2.INTER_CUBIC)
# b = cv2.resize(a, [256, 256], interpolation = cv2.INTER_CUBIC)
# b = caffe.io.resize_image(a, [256, 256])
print b.shape
print b.dtype
print image_label[1][10]
print image_label[1].dtype
print '{:0>8d}'.format(2000)
print 'batch {} writen'.format(20)

uint8
int64
(3, 32, 32)
(3, 256, 256)
int64
0
uint8
00002000
batch 20 writen


In [34]:
def data_bin_to_lmdb(lmdb_file, batch_size, data_num):
    # create the leveldb file
    lmdb_env = lmdb.open(lmdb_file, map_size=int(1e12))#生成一个数据文件，定义最大空间
    lmdb_txn = lmdb_env.begin(write=True)              #打开数据库的句柄
    datum = caffe_pb2.Datum()                          #这是caffe中定义数据的重要类型
    im_re = np.zeros((3, 256, 256)).astype('int')
    
    n = 1
    x = 0
    while n <= data_num:
        read_count = 10000
        start_location = 0
#         image_label = get_image_label('data/cifar10/data_batch_' + str(n) + '.bin', read_count, start_location)
        image_label = get_image_label('data/cifar10/test_batch.bin', read_count, start_location)
        for i in range(len(image_label[0])):
            x += 1
            im = image_label[0][i]
            for c in range(3):
                im_re[c] = cv2.resize(im[c], (256, 256), interpolation = cv2.INTER_CUBIC)
                    
            label = image_label[1][i]                        #图像的标签，为了方便存储，这个必须是整数。
            datum = caffe.io.array_to_datum(im_re, label)    #将数据以及标签整合为一个数据项

            
            keystr = '{:0>8d}'.format(x - 1)                 #lmdb的每一个数据都是由键值对构成的，因此生成一个用递增顺序排列的定长唯一的key
            lmdb_txn.put(keystr, datum.SerializeToString())  #调用句柄，写入内存。
            
            # write batch
            if x % batch_size == 0:                          #每当累计到一定的数据量，便用commit方法写入硬盘。
                lmdb_txn.commit()
                lmdb_txn = lmdb_env.begin(write = True)      #commit之后，之前的txn就不能用了，必须重新开一个。
                print 'batch {} writen'.format(x)
        n += 1
    lmdb_env.close()                                         #结束后记住释放资源，否则下次用的时候打不开。
    
    
#basic setting
lmdb_file = 'examples/cifar10/cifar10_test_lmdb_256x256'   #期望生成的数据文件
# lmdb_file = 'examples/cifar10/cifar10_train_lmdb_256x256'
batch_size = 20             #lmdb对于数据进行的是先缓存后一次性写入从而提高效率，因此定义一个batch_size控制每次写入的量。
data_num = 1
data_bin_to_lmdb(lmdb_file, batch_size, data_num)



batch 20 writen
batch 40 writen
batch 60 writen
batch 80 writen
batch 100 writen
batch 120 writen
batch 140 writen
batch 160 writen
batch 180 writen
batch 200 writen
batch 220 writen
batch 240 writen
batch 260 writen
batch 280 writen
batch 300 writen
batch 320 writen
batch 340 writen
batch 360 writen
batch 380 writen
batch 400 writen
batch 420 writen
batch 440 writen
batch 460 writen
batch 480 writen
batch 500 writen
batch 520 writen
batch 540 writen
batch 560 writen
batch 580 writen
batch 600 writen
batch 620 writen
batch 640 writen
batch 660 writen
batch 680 writen
batch 700 writen
batch 720 writen
batch 740 writen
batch 760 writen
batch 780 writen
batch 800 writen
batch 820 writen
batch 840 writen
batch 860 writen
batch 880 writen
batch 900 writen
batch 920 writen
batch 940 writen
batch 960 writen
batch 980 writen
batch 1000 writen
batch 1020 writen
batch 1040 writen
batch 1060 writen
batch 1080 writen
batch 1100 writen
batch 1120 writen
batch 1140 writen
batch 1160 writen
batch 118

batch 9200 writen
batch 9220 writen
batch 9240 writen
batch 9260 writen
batch 9280 writen
batch 9300 writen
batch 9320 writen
batch 9340 writen
batch 9360 writen
batch 9380 writen
batch 9400 writen
batch 9420 writen
batch 9440 writen
batch 9460 writen
batch 9480 writen
batch 9500 writen
batch 9520 writen
batch 9540 writen
batch 9560 writen
batch 9580 writen
batch 9600 writen
batch 9620 writen
batch 9640 writen
batch 9660 writen
batch 9680 writen
batch 9700 writen
batch 9720 writen
batch 9740 writen
batch 9760 writen
batch 9780 writen
batch 9800 writen
batch 9820 writen
batch 9840 writen
batch 9860 writen
batch 9880 writen
batch 9900 writen
batch 9920 writen
batch 9940 writen
batch 9960 writen
batch 9980 writen
batch 10000 writen
